Miniproject 1

Problem
- Wavelength in mircometers
- Distance from planet center
- Cross-section the data file 
- Parameters from the Data: planet mass, planet radius, surface density, temperature
- Optical depth
- Transmissivity

Need to do
- At least one class (Maybe three easier and more interchangble for Parameters, Cross section and Optical Depth calculator)
- Read parameters from a parameter file (use jason and dictionary)
- Create a module that can be imported

Formula 1 Scale Height
-H = R_g × T_atm / (m_gas × g)
- R_g = 8.314 J/(mol·K) Universal gas constant
- T_atm = Atmospheric temperature K
- m_gas = Molecular mass of gas CO2 = 44.01 g/mol
- g = Surface gravity m/s^2

Formula 2 Surface Gravity
g = G × M_planet / R_planet^2
- G = 6.674×10^-11 m^3/(kg·s^2) Gravitational constant
- M_planet = Planet mass kg
- R_planet = Planet radius m

Formula 3 Number Density
(R) = N_surf × exp(-h/H)
- N_surf = Surface number density[molecules/m^3
- h = R - R_planet altitude m
- H = Scale height m

Formula 4 Integrated Column Density
Exact formula:
N_f(R) = 2 × N(R) × R × exp(R/H) × K1(R/H)    
Approximation (for large R/H):
N_f(R) ≈ N(R) × √(2πRH)

Formula 5 Optical Depth
τ(λ) = N_f × σ(λ)
- N_f = Integrated column density molecules/m^2
- σ(λ) = Cross-section at wavelength λ m^2/molecule

Formula 6 Transmissivity
T(λ) = exp(-τ(λ))

Data File extraction 
- Extract info pressure, temperature, molecule name
- Divide the columns nu and sigma
- wavenumber, cross-section, k

How to do Class structure
1.
Atmospheric Parameters (Store/load all parameters)
2.
Cross Section Data (Load and Interpolate σ(ν))
3.
Optical Depth Calculator 

Module Functions:
- initialize -> Load all data
- opticaldept (λ, R) -> (τ, T)
- get_scale_height -> H
- get_column_density(R) -> N(R)


Checklist for classes

Class Atmospheric Parameters 
- Creat parameter file (JSON Format) (Use JSON for intercahngibilty)
- Read the File
- Define the Parameters
- Read from JSON
- Define Values with JSON Values

Class Cross section Data
- Load data for interpolating
- Header Pressure, Temperature, Number of molecules
- Column 1: wavenumber in cm^-1
- Column 2: cross-section in cm^2/molecule
- Column 3: k in m^-1 
- Change units of the Data
- Cross section

Class Optical Depth Calculator
- Scale height calculation
- Column density approximation
- Optical depth
- Transmissivity
- Define Funcion

In [1]:
import json
import numpy as np
from scipy.special import k1
from scipy.interpolate import interp1d
 
 
class AtmosphericParameters:  # Class for atmospheric and planetary parameters
 
    def __init__(self):  # Initialize parameters with 0
        self.wavelength_um = 0
        self.radius_R = 0
        self.cross_section_file = 0
        self.density_surf = 0
        self.temperature_atm = 0
        self.planet_mass = 0
        self.planet_radius = 0
 
    def load_from_file(self, filepath):  # Load parameters from a JSON file
        with open(filepath, 'r') as f:
            params = json.load(f)
 
        # Assign parameters individually 
        self.wavelength_um = params.get('wavelength_um')
        self.radius_R = params.get('radius_R')
        self.cross_section_file = params.get('cross_section_file')
        self.density_surf = params.get('density_surf')
        self.temperature_atm = params.get('temperature_atm')
        self.planet_mass = params.get('planet_mass')
        self.planet_radius = params.get('planet_radius')
 
        return params
 
    def to_dict(self):
        # Parameters as a dictionary
        return dict(self.__dict__)
 
 
class CrossSectionData:  # Class for cross-section and interpolation
 
    def __init__(self):  # Initialize cross-section data storage
        self.wavenumbers = 0
        self.cross_sections = 0
        self.interpolator = 0
        self.pressure = 0
        self.temperature = 0
        self.molecule_name = 0
 
    def load_data(self, filepath):  # Load cross-section data from file.
        # - Header with Pressure, Temperature, Number of molecules
        # - Data rows with 3 columns
        wavenumbers = []
        cross_sections = []
 
        with open(filepath, 'r') as f:
            lines = f.readlines()
 
        i = 0
        while i < len(lines):
            line = lines[i].strip()
 
            if line.startswith('Pressure'):
                i += 1
                self.pressure = float(lines[i].strip())
 
            elif line.startswith('Temperature'):
                i += 1
                self.temperature = float(lines[i].strip())
 
            elif line.startswith('Name:'):
                i += 1
                self.molecule_name = lines[i].strip()
 
            elif 'nu' in line and 'sigma' in line:
                i += 1
                break
 
            i += 1
 
        while i < len(lines):
            line = lines[i].strip()
 
            if not line or line.startswith('#'):
                i += 1
                continue
 
            parts = line.split()
            if len(parts) >= 2:
                wavenumbers.append(float(parts[0]))
                cross_sections.append(float(parts[1]))
 
            i += 1
 
        self.wavenumbers = np.array(wavenumbers)
        self.cross_sections = np.array(cross_sections)
 
        print("Loaded cross-section data")
        print("Molecule:", self.molecule_name)
        print("Pressure:", f"{self.pressure}", "atm")
        print("Temperature:", f"{self.temperature}", "K")
        print("Data points:", len(self.wavenumbers))
 
        self.interpolator = interp1d(
            self.wavenumbers,
            self.cross_sections,
            kind='linear',
            bounds_error=False,
            fill_value='extrapolate'
        )
 
    def wavelength_to_wavenumber(self, wavelength_um):  # Wavelength in micrometers to wavenumber
        return 1e4 / wavelength_um
 
    def wavenumber_to_wavelength(self, wavenumber):  # Wavenumber to wavelength in micrometers
        return 1e4 / wavenumber
 
    def get_cross_section(self, wavelength_um):  # Cross-section for a given wavelength
        wn = self.wavelength_to_wavenumber(wavelength_um)
        return float(self.interpolator(wn))
 
    def get_cross_section_at_wavenumber(self, wavenumber):  # Cross-section for a given wavenumber
        return float(self.interpolator(wavenumber))
 
 
class OpticalDepthCalculator:  # Main class for optical depth and transmissivity
 
    R_g = 8.314
    m_CO2 = 44.01e-3
    G = 6.674e-11
 
    def __init__(self, parameters, cross_section_data):  # Initialize parameters and cross-section data
        self.params = parameters
        self.cross_section = cross_section_data
 
        self.g = self._calculate_gravity()
        self.scale_height = self._calculate_scale_height()
 
    def _calculate_gravity(self):  # Gravitational acceleration at planet surface
        return self.G * self.params.planet_mass / (self.params.planet_radius ** 2)
 
    def _calculate_scale_height(self):  # Atmospheric scale height
        return self.R_g * self.params.temperature_atm / (self.m_CO2 * self.g)
 
    def calculate_column_density_N(self, R):  # Number density
        h = R - self.params.planet_radius
        if h < 0:
            raise ValueError("Fehler")
 
        return self.params.density_surf * np.exp(-h / self.scale_height)
 
    def calculate_column_density_integrated(self, R, use_approximation=True):  # Integrated column density
        N_R = self.calculate_column_density_N(R)
        H = self.scale_height
 
        if use_approximation:
            return N_R*np.sqrt(2*np.pi*R*H)
        else:
            x = R / H
            return 2 *N_R* R*np.exp(x)*k1(x)
 
    def calculate_optical_depth(self, wavelength_um, R):  # Optical depth
        sigma_cm2 = self.cross_section.get_cross_section(wavelength_um)
        sigma_m2 = sigma_cm2 * 1e-4
 
        N_f = self.calculate_column_density_integrated(R)
        return N_f * sigma_m2
 
    def calculate_transmissivity(self, wavelength_um, R):  # Transmissivity at distance
        tau = self.calculate_optical_depth(wavelength_um, R)
        return np.exp(-tau)
 
 
params = None
cross_section_data = None
calculator = None
 
 
def initialize(param_file='parameters.json'):  # Module with a parameter file
    global params, cross_section_data, calculator
 
    params = AtmosphericParameters()
    params.load_from_file(param_file)
 
    cross_section_data = CrossSectionData()
    cross_section_data.load_data(params.cross_section_file)
 
    calculator = OpticalDepthCalculator(params, cross_section_data)
 
 
def opticaldepth(wavelength_um, R):
    global calculator
    
    if calculator is None:
        raise RuntimeError("Error")
    
    tau = calculator.calculate_optical_depth(wavelength_um, R)
    return tau, np.exp(-tau) 
 
 
def get_scale_height():  # Atmospheric scale height in meters
    if calculator is None:
        raise RuntimeError("Error")
    return calculator.scale_height
 
 
def get_surface_gravity():  # Return the surface gravity
    if calculator is None:
        raise RuntimeError("Error")
    return calculator.g
 
 
def get_column_density(R):  # Number density at distance R from planet center
    if calculator is None:
        raise RuntimeError("Error")
    return calculator.calculate_column_density_N(R)
 
 
def get_integrated_column_density(R, use_approximation=True):  #Integrated column density at distance R
    if calculator is None:
        raise RuntimeError("Error")
    return calculator.calculate_column_density_integrated(R, use_approximation)
 
 
if __name__ == "__main__":
    import os
 
    print("Optical Depth Calculator")
 
    try:
        script_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        script_dir = os.getcwd()
 
    param_file = os.path.join(script_dir, 'parameters.json')
    initialize(param_file)
    print(f"\nPlanetary Parameters:")
    print(f"Planet mass: {params.planet_mass} kg")
    print(f"Planet radius: {params.planet_radius} m")
    print(f"Surface temperature: {params.temperature_atm} K")
    print(f"Surface density: {params.density_surf} molecules/m³")
    print(f"\nDerived Quantities:")
    print(f"Surface gravity: {get_surface_gravity()} m/s²")
    print(f"Scale height: {get_scale_height()/1000} km")
    R = params.planet_radius  # Start at surface
    print(f"\nCalculation at surface:")
    print(f"Distance R: {R} m")
    # Number density at R
    N_R = get_column_density(R)
    print(f"Number density N(R): {N_R} molecules/m³")
    # Cross-section
    wavelength = 20.0
    sigma = cross_section_data.get_cross_section(wavelength)
    print(f"Cross-section at {wavelength} μm: {sigma} cm²/molecule")
    # Optical depth and transmissivity
    tau, T = opticaldepth(wavelength, R)
    print(f"\nOptical depth τ: {tau}")
    print(f"Transmissivity T: {T}")

Optical Depth Calculator
Loaded cross-section data
Molecule: CO2
Pressure: 1e-06 atm
Temperature: 200.0 K
Data points: 951549

Planetary Parameters:
Planet mass: 5.972e+24 kg
Planet radius: 6371000.0 m
Surface temperature: 200.0 K
Surface density: 1e+19 molecules/m³

Derived Quantities:
Surface gravity: 9.819532032815959 m/s²
Scale height: 3.8476703444965734 km

Calculation at surface:
Distance R: 6371000.0 m
Number density N(R): 1e+19 molecules/m³
Cross-section at 20.0 μm: 1.54724228332271e-29 cm²/molecule

Optical depth τ: 6.072268827771693e-09
Transmissivity T: 0.9999999939277312


This is Data for Earth
